# Projeto Final 

## Setup

In [ ]:
from pyspark.sql import functions as sf
import pyspark.sql.functions as f

# Modeling
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
## Bibliotecas Gráficas
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[4]") \
            .appName("nyc_caio_garcia") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,com.microsoft.azure:azure-storage:8.6.6") \
            .getOrCreate()

In [ ]:
# Acesso aos dados na nuvem
STORAGE_ACCOUNT = 'dlspadseastusprod'
CONTAINER = 'big-data-comp-nuvem'
FOLDER = 'airline-delay'
TOKEN = 'lSuH4ZI9BhOFEhCF/7ZQbrpPBIhgtLcPDfXjJ8lMxQZjaADW4p6tcmiZGDX9u05o7FqSE2t9d2RD+ASt0YFG8g=='

spark.conf.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)

### Schema
Schema definido de acordo com o dicionário de dados em `projeto_final_dicionário.xlsx`

In [ ]:
from pyspark.sql.types import *

labels = (('FL_DATE', TimestampType()),
          ('OP_CARRIER', StringType()),
          ('OP_CARRIER_FL_NUM', IntegerType()),
          ('ORIGIN', StringType()),
          ('DEST', StringType()),
          ('CRS_DEP_TIME', IntegerType()),
          ('DEP_TIME', FloatType()),
          ('DEP_DELAY', FloatType()),
          ('TAXI_OUT', FloatType()),
          ('WHEELS_OFF', FloatType()),
          ('WHEELS_ON', FloatType()),
          ('TAXI_IN', FloatType()),
          ('CRS_ARR_TIME', IntegerType()),
          ('ARR_TIME', FloatType()),
          ('ARR_DELAY', FloatType()),
          ('CANCELLED', FloatType()),
          ('CANCELLATION_CODE', StringType()),
          ('DIVERTED', FloatType()),
          ('CRS_ELAPSED_TIME', FloatType()),
          ('ACTUAL_ELAPSED_TIME', FloatType()),
          ('AIR_TIME', FloatType()),
          ('DISTANCE', FloatType()),
          ('CARRIER_DELAY', FloatType()),
          ('WEATHER_DELAY', FloatType()),
          ('NAS_DELAY', FloatType()),
          ('SECURITY_DELAY', FloatType()),
          ('LATE_AIRCRAFT_DELAY', StringType()))

schema = StructType([StructField(x[0], x[1], True) for x in labels])

In [ ]:
# Columns with values in minutes
minute_columns = ["TAXI_OUT","TAXI_IN","DEP_DELAY","ARR_DELAY","AIR_TIME","CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME",
                  "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]

# Subset from minute columns with no data leak from moment of take off
clean_min_columns = ["TAXI_OUT", "DEP_DELAY", "CRS_ELAPSED_TIME"]

# Columns with time information on format 'hhmm'
# Not proper for numerical manipulation
odd_format_columns = ["CRS_DEP_TIME","DEP_TIME","WHEELS_OFF","WHEELS_ON","ARR_TIME","CRS_ARR_TIME"]

### Carregamento de dados
Dados carregados da nuvem como spark data frame

Exemplo para o ano de 2009:

In [ ]:
config = spark.sparkContext._jsc.hadoopConfiguration()
config.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)
sc = spark.sparkContext

df_exemple = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2009.csv"\
                  .format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema=schema)
df_exemple.take(2)

Como temos dados para os anos de 2009 até o ano de 2018. Iremos criar um dicionario contendo os dataframes de cada ano separadamente. 

In [ ]:
# Criando dicionario de dataframes
df_for_year = {}

# Loop lendo arquivo de cada ano e salvando no dicionario
for year in range(2009, 2019):
    # Ajustando o caminho
    file_path = "wasbs://{}@{}.blob.core.windows.net/{}/{}.csv"\
                    .format(CONTAINER, STORAGE_ACCOUNT, FOLDER, year)
    
    # lendo arquivo csv 
    df_name = "df_{}".format(year)
    df = spark.read.csv(file_path, header=True, schema=schema)
    
    # Adicionando df ao dicionario de dataframes 
    df_for_year[df_name] = df
    

# Visualizando as primeiras linhas de 2012
df_for_year["df_2012"].take(5)

Com todos os dataframes pré-importados(lazy) podemos realizar um merge unindo todos os anos.

In [ ]:
# Importando função reduce para realizar o merge
from functools import reduce 

In [ ]:
# União de todos os DataFrames em um único Data frame
df_final = reduce(sf.DataFrame.union, df_for_year.values())

# Criando a coluna "year" baseada na coluna "date"
df_final = df_final.withColumn("year", sf.year("FL_DATE"))

# Exibindo as primeiras linhas
df_final.take(10)

Assim, temos o dataframe com todos os anos.

Criando um cahce do df_final para acelerar o processamento dos dados.

In [ ]:
df_final.cache()

In [ ]:
OBSERVACOES = df_final.count()
assert (OBSERVACOES == 61556964)

In [ ]:
OBSERVACOES

A base contem 61556964 observações.

In [ ]:
CANCELAMENTOS = df_final.filter(df_final.CANCELLED == 1).count()
assert (CANCELAMENTOS == 973209)

In [ ]:
CANCELAMENTOS

Dos voos na base, 973209 foram cancelados.

## Tratamento de dados faltantes

Nesta etapa será realizada uma ánalise de valores nulos em cada variavel preditora.

### Resumo Geral

Primeiramente iremos ánalisar a quantidade de valores nulos por coluna.

In [ ]:
# Agrupando e calculando o numero de valores nulos por coluna
missing_counts = df_final.select([sf.col(column).isNull().cast("int").alias(column) for column in df_final.columns]) \
                         .groupBy() \
                         .sum()

In [ ]:
# Criando dataframe de colunas com valores zerados
missing_counts_df = missing_counts.toPandas().transpose()

# Filtrando apenas colunas com valores nulos
missing_counts_df = missing_counts_df[missing_counts_df[0]>0]

# Renomeando a coluna
missing_counts_df = missing_counts_df.rename(columns={0:"nulos"})

In [ ]:
# Contando número de colunas com valores nulos
print("Número de colunas com valores faltantes:")
missing_counts_df.count()[0]

In [ ]:
# Cálculando porcentagem de valores faltantes
missing_counts_df["%nulos"] = (missing_counts_df["nulos"]/OBSERVACOES) * 100

# Ordenando por % de nulos
missing_counts_df = missing_counts_df.sort_values("%nulos", ascending=False)

# Visualizando resultados
missing_counts_df

In [ ]:
# Visualizando missing em gráfico de barras
# Ajustando o tamanho da figura
plt.figure(figsize=(10, 4))

# Plotando o gráfico de barras
sns.barplot(x=missing_counts_df.index, y=missing_counts_df["%nulos"], color="red")

# Adicionando inclinação aos valores do eixo x
plt.xticks(rotation=45, ha='right')

# Adicionando título e rótulos aos eixos
plt.title('Gráfico de Barras')
plt.xlabel("Colunas")
plt.ylabel("%Nulos")

# Exibindo o gráfico
plt.show()

Conforme evidenciado no gráfico apresentado, nota-se que a coluna "CANCELLATION_CODE" exibe uma lacuna em praticamente 100% dos dados, enquanto as colunas "LATE_AIRCRAFT_DELAY", "NAS_DELAY", "WEATHER_DELAY", "CARRIER_DELAY" e "SECURITY_DELAY" apresentam uma ausência de informações em torno de 80%.

Das 27 colunas na base de dados, 19 tem valores faltantes. Para compreender melhor o impacto desses valores nulos, iremos dividir esta ánalise em grupos: _Cancelamentos_, _Voo_ e _Atrasos_

### Cancelamentos

Dentre as colunas que indicam se o voo foi cancelado, temos a coluna CANCELLATION_CODE e CANCELLED. Nesta etapa iremos verificar se essas volunas possuem valores nulos iguais, em outras palavras, se o código descrito na coluna CANCELLATION_CODE corresponde aos voos demarcados como cancelados na coluna CANCELLED.

In [ ]:
try:
    assert (df_final.filter((df_final.CANCELLATION_CODE.isNull()) &
                            (df_final.CANCELLED == 0)).count() ==
            df_final.filter(df_final.CANCELLED == 0).count())
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

Todos os valores faltantes de CANCELLATION_CODE são referentes a voos que não foram cancelados.

### Voo

O grupo _Voo_ apresenta uma relação entre voos cancelados e as 7 variáveis:
 - DEP_TIME
 - DEP_DELAY
 - TAXI_OUT
 - WHEELS_OFF
 - WHEELS_ON
 - TAXI_IN
 - ARR_TIME

#### Testes

`DEP_TIME` e `DEP_DELAY`: co-ausentes, em todos os voos cancelados

O primeiro teste realizado será conferindo se as colunas DEP_TIME, DEP_DELAY, TAXI_OUT, WHEELS_OFF, WHEELS_ON, TAXI_IN e ARR_TIME estão contidas dentro da coluna DEP_TIME.

In [ ]:
try:
    assert (
        df_final.filter(
            (df_final.DEP_TIME.isNull()) &
            (df_final.DEP_DELAY.isNull()) &
            (df_final.TAXI_OUT.isNull()) &
            (df_final.WHEELS_OFF.isNull()) &
            (df_final.WHEELS_ON.isNull()) &
            (df_final.TAXI_IN.isNull()) &
            (df_final.ARR_TIME.isNull())
        ).count() ==
        df_final.filter(df_final.DEP_TIME.isNull()).count()
    )
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

Ao rodar o código acima, concluimos que essas colunas estão contidas na coluna DEP_TIME.

O próximo teste será com relação a coluna DEP_TIME e DEP_DELAY correspondem com relação ao número de linhas nulas.

In [ ]:
try:
    assert (df_final.filter((df_final.DEP_TIME.isNull())   &
                            (df_final.DEP_DELAY.isNull())).count() ==
            df_final.filter(
                df_final.DEP_TIME.isNull()).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.DEP_TIME.isNull())   &
                            (df_final.CANCELLED == 1)).count() ==
            df_final.filter(
                df_final.DEP_TIME.isNull()).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")

`TAXI_OUT` e `WHEELS_OFF`: co-ausentes e cancelados

In [ ]:
try:
    assert (df_final.filter(
                  (df_final.TAXI_OUT.isNull())   &
                  (df_final.WHEELS_OFF.isNull()) &
                  (df_final.WHEELS_ON.isNull())  &
                  (df_final.TAXI_IN.isNull())    &
                  (df_final.ARR_TIME.isNull())).count() ==
            df_final.filter(
                df_final.WHEELS_OFF.isNull()).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.TAXI_OUT.isNull())   &
                            (df_final.WHEELS_OFF.isNull())).count() ==
        df_final.filter(
            df_final.WHEELS_OFF.isNull()).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")


In [ ]:
try:
    assert (df_final.filter((df_final.TAXI_OUT.isNull())   &
                            (df_final.CANCELLED == 1)).count() ==
       df_final.filter(df_final.TAXI_OUT.isNull()).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")

`WHEELS_ON`, `TAXI_IN` e `ARR_TIME`: co-ausentes

In [ ]:
try:
    assert (df_final.filter((df_final.WHEELS_ON.isNull())  &
                            (df_final.TAXI_IN.isNull())    &
                            (df_final.ARR_TIME.isNull())).count() ==
        df_final.filter(
            df_final.TAXI_IN.isNull()).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.TAXI_IN.isNull())   &
                            (df_final.CANCELLED == 1)).count() ==
        df_final.filter(
            df_final.CANCELLED == 1).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.TAXI_IN.isNull()) &
                            (df_final.CANCELLED == 0)   &
                            (df_final.DIVERTED == 1)).count() ==
        df_final.filter(
            (df_final.TAXI_IN.isNull()) &
            (df_final.CANCELLED == 0)).count())
    
    print("Igualdade atendida")
    
except AssertionError:
    print("Igualdade não atendida")

#### Análise

O grupo _Voo_ apresenta uma relação entre voos cancelados e as 7 variáveis:
 - DEP_TIME
 - DEP_DELAY
 - TAXI_OUT
 - WHEELS_OFF
 - WHEELS_ON
 - TAXI_IN
 - ARR_TIME
 
Os valores faltantes para WHEELS_ON, TAXI_IN e ARR_TIME coincidem nas mesmas observações (com uma exceção descrita mais abaixo). Todos os voos cancelados se encontram dentre essas observações. Os valores faltantes para TAXI_OUT e WHEELS_OFF coincidem nas mesmas observações, todas referentes a voos cancelados. Finalmente, os valores faltantes de DEP_TIME e DEP_DELAY coincidem nas mesmas observações, todas com valores faltantes para TAXI_OUT.

Todos os voos que não foram cancelados mas não tem informação da hora de aterrisagem (`(df.WHEELS_ON.isNull()) & (df.CANCELLED == 0)`) foram redirecionados para um aeroporto diferente do aeroporto destino original (`df.DIVERTED == 1`)

Destas relações, supomos:
 - A diferença entre DEP_TIME e WHEELS_OFF pode ser devido a voos que chegam a sair do chão antes de serem cancelados, e voos que são cancelados após o embarque mas antes da decolagem.
 - Nenhum desses valores faltantes parece implausível o suficiente para assumirmos erro nos dados baseado apenas nessa análise. Alguns desses dados podem vir a ser retirados mesmo assim por questão de propriedades dos algorítmos utilizados mais a frente.

### Atrasos

#### Testes

In [ ]:
try:
    assert (df_final.filter((df_final.CANCELLED == 0) &
                  (df_final.DEP_DELAY > 0)).count() == 
        22280599)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.CANCELLED == 0) &
                  (df_final.ARR_DELAY > 0)).count() ==
        22563856)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")


In [ ]:
try:
    assert (df_final.filter((df_final.CANCELLED == 0) &
                  ((df_final.DEP_DELAY > 0) |
                   (df_final.ARR_DELAY > 0))).count() ==
        28817028)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")


In [ ]:
try:
    assert (OBSERVACOES - df_final.filter(df_final.CARRIER_DELAY.isNull()).count() ==
        11390740)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.CARRIER_DELAY.isNull()) &
                  (df_final.WEATHER_DELAY.isNull()) &
                  (df_final.NAS_DELAY.isNull()) &
                  (df_final.SECURITY_DELAY.isNull()) &
                  (df_final.LATE_AIRCRAFT_DELAY.isNull())).count() ==
        df_final.filter(df_final.CARRIER_DELAY.isNull()).count())
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
df_final.filter(df_final.CARRIER_DELAY == 0).select(df_final.OP_CARRIER_FL_NUM, df_final.CARRIER_DELAY).take(10)

#### Análise

Todos os dados faltantes referentes a categoria de atraso coincidem nas mesmas observações.

Há menos observações com informação sobre a causa do atraso do que voos atrasados, independente se medindo o atraso de saída ou de chegada.

### Anomalia

In [ ]:
try:
    assert (df_final.filter((df_final.ACTUAL_ELAPSED_TIME.isNull()) &
                  (df_final.AIR_TIME == 0)                &
                  (df_final.WHEELS_ON.isNull())           &
                  (df_final.ARR_TIME.isNull())            &
                  (df_final.ARR_DELAY == 0)               &
                  (df_final.TAXI_IN == 0)                 &
                  (df_final.CANCELLED == 0)).count() == 1)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

Uma mesma observação é responsavel pela discrepância na quantidade total de valores faltantes entre WHEELS_ON, TAXI_IN e ARR_TIME, e ARR_DELAY, ACTUAL_ELAPSED_TIME e AIR_TIME.
Um valor de `AIR_TIME == 0` nao faz sentido para um voo que não foi cancelado, e o mesmo se aplica a `TAXI_IN == 0`. Ao retirar essa observação da base, a análise de dados faltantes por grupo torna-se mais consistente.

## Consistência
Conferindo que os dados disponíveis de fato seguem as relações definidas no dicionário de dados.

In [ ]:
try:
    assert (df_final.filter(df_final.AIR_TIME +
                            df_final.TAXI_IN +
                            df_final.TAXI_OUT !=
                            df_final.ACTUAL_ELAPSED_TIME).count() == 0)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.CANCELLED == 1) &
                            (df_final.DIVERTED == 1)).count() == 0)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

In [ ]:
try:
    assert (df_final.filter((df_final.DEP_TIME % 1 != 0) | (df_final.DEP_DELAY % 1 != 0)).count() == 0)
    print("Igualdade atendida")
except AssertionError:
    print("Igualdade não atendida")

# Análise Exploratória

Nesta etapa foi realizada uma análise exploratória dos dados que entraram na modelagem, dentre elas 4 categoricas e 4 númericas.

Preditoras Númericas
- TAXI_OUT
- DEP_DELAY
- CRS_ELAPSED_TIME
- DISTANCE

Preditoras Categóricas:
- OP_CARRIER
- OP_CARRIER_FL_NUM
- ORIGIN
- DEST

### Númericas

Nesta etapa iresmo analisar a influência e correlação entre as váriaveis númericas e a váriavel target. Como estamos trabalhando com um número elevado de linhas, não é possivel utilizar uma amostra muito significativa dos dados para plotar os gráficos de dispersão. Para contornar este problema estamos considerando 6 amostras com 5% da base e seeds distintos para plotar o gráfico de dispersão e entender a relação desta variavel 

In [ ]:
# Criando dataframe com as duas váriaveis
df_taxi_out_delay = df_final.select("TAXI_OUT", "ARR_DELAY")

# Lista de sementes aleatórias
sementes_aleatorias = [42, 123, 567, 789, 999, 111]

# Configurar subplots
num_linhas = int(len(sementes_aleatorias) / 3)
num_colunas = 3  # Você pode ajustar o número de colunas conforme necessário
fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(12,8))

# Loop através das sementes
for i, semente in enumerate(sementes_aleatorias):
    # Criar uma amostra do DataFrame usando a semente
    df_amostra = df_taxi_out_delay.sample(withReplacement=False, fraction=0.05, seed=semente)

    # Coletar a amostra para o ambiente local (Pandas DataFrame)
    df_pandas = df_amostra.toPandas()

    # Realizar análise exploratória com Seaborn e Matplotlib
    # Converter axs[i] em um eixo individual usando [0]
    sns.scatterplot(x='TAXI_OUT', y='ARR_DELAY', data=df_pandas, ax=axs[i // num_colunas, i % num_colunas])
    axs[i // num_colunas, i % num_colunas].set_xlabel('TAXI_OUT')
    axs[i // num_colunas, i % num_colunas].set_ylabel('ARR_DELAY')
    axs[i // num_colunas, i % num_colunas].set_title(f'Gráfico de Dispersão - Semente: {semente}')

# Ajustar layout e exibir os subplots
plt.tight_layout()
plt.show()

Observando os resultados dos gráficos acima, fica evidente que não há um padrão linear entre o intervalo de tempo desde a saída do avião do portão até a decolagem e o atraso na aterrissagem. No entanto, é importante notar que não foram identificados atrasos significativos mesmo quando esse período de tempo até a decolagem é ampliado.

Realizando a ánalise com a segunda variável.

In [ ]:
# Criando dataframe com as duas váriaveis
df_dep_delay_delay = df_final.select("DEP_DELAY", "ARR_DELAY")

# Lista de sementes aleatórias
sementes_aleatorias = [42, 123, 567, 789, 999, 111]

# Configurar subplots
num_linhas = int(len(sementes_aleatorias) / 3)
num_colunas = 3  # Você pode ajustar o número de colunas conforme necessário
fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(12,8))

# Loop através das sementes
for i, semente in enumerate(sementes_aleatorias):
    # Criar uma amostra do DataFrame usando a semente
    df_amostra = df_dep_delay_delay.sample(withReplacement=False, fraction=0.05, seed=semente)

    # Coletar a amostra para o ambiente local (Pandas DataFrame)
    df_pandas = df_amostra.toPandas()

    # Realizar análise exploratória com Seaborn e Matplotlib
    # Converter axs[i] em um eixo individual usando [0]
    sns.scatterplot(x='DEP_DELAY', y='ARR_DELAY', data=df_pandas, ax=axs[i // num_colunas, i % num_colunas], color="orange")
    axs[i // num_colunas, i % num_colunas].set_xlabel('DEP_DELAY')
    axs[i // num_colunas, i % num_colunas].set_ylabel('ARR_DELAY')
    axs[i // num_colunas, i % num_colunas].set_title(f'Gráfico de Dispersão - Semente: {semente}')

# Ajustar layout e exibir os subplots
plt.tight_layout()
plt.show()

Como pode ser observado, existe uma relação linear quase perfeita entre o atraso na decolagem e o atraso na aterrissagem dos voos analisados. Em outras palavras, quanto maior for o atraso na decolagem, maior será o atraso na chegada do avião ao seu destino. Isso era esperado, no entanto, vale a pena notar que, em algumas subdivisões de seeds, voos sem atraso na decolagem também apresentaram atrasos na aterrissagem. Isso sugere que outros fatores também podem influenciar a variável alvo.

In [ ]:
# Criando dataframe com as duas váriaveis
df_elapsed_time_delay = df_final.select("CRS_ELAPSED_TIME", "ARR_DELAY")

# Lista de sementes aleatórias
sementes_aleatorias = [42, 123, 567, 789, 999, 111]

# Configurar 
num_linhas = int(len(sementes_aleatorias) / 3)
num_colunas = 3  # Você pode ajustar o número de colunas conforme necessário
fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(12,8))

# Loop através das sementes
for i, semente in enumerate(sementes_aleatorias):
    # Criar uma amostra do DataFrame usando a semente
    df_amostra = df_elapsed_time_delay.sample(withReplacement=False, fraction=0.05, seed=semente)

    # Coletar a amostra para o ambiente local (Pandas DataFrame)
    df_pandas = df_amostra.toPandas()

    # Realizar análise exploratória com Seaborn e Matplotlib
    # Converter axs[i] em um eixo individual usando [0]
    sns.scatterplot(x='CRS_ELAPSED_TIME', y='ARR_DELAY', data=df_pandas, ax=axs[i // num_colunas, i % num_colunas], color="red")
    axs[i // num_colunas, i % num_colunas].set_xlabel('CRS_ELAPSED_TIME')
    axs[i // num_colunas, i % num_colunas].set_ylabel('ARR_DELAY')
    axs[i // num_colunas, i % num_colunas].set_title(f'Gráfico de Dispersão - Semente: {semente}')

# Ajustar layout e exibir os subplots
plt.tight_layout()
plt.show()

Obsevando o resultado dos gráficos acima, não é possivel perceber nenhuma correlação evidente entre as duas variaveis analisadas.

In [ ]:
# Criando dataframe com as duas váriaveis
df_distance_delay = df_final.select("DISTANCE", "ARR_DELAY")

# Lista de sementes aleatórias
sementes_aleatorias = [42, 123, 567, 789, 999, 111]

# Configurar 
num_linhas = int(len(sementes_aleatorias) / 3)
num_colunas = 3  # Você pode ajustar o número de colunas conforme necessário
fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(12,8))

# Loop através das sementes
for i, semente in enumerate(sementes_aleatorias):
    # Criar uma amostra do DataFrame usando a semente
    df_amostra = df_distance_delay.sample(withReplacement=False, fraction=0.05, seed=semente)

    # Coletar a amostra para o ambiente local (Pandas DataFrame)
    df_pandas = df_amostra.toPandas()

    # Realizar análise exploratória com Seaborn e Matplotlib
    # Converter axs[i] em um eixo individual usando [0]
    sns.scatterplot(x='DISTANCE', y='ARR_DELAY', data=df_pandas, ax=axs[i // num_colunas, i % num_colunas], color="green")
    axs[i // num_colunas, i % num_colunas].set_xlabel('CRS_ELAPSED_TIME')
    axs[i // num_colunas, i % num_colunas].set_ylabel('ARR_DELAY')
    axs[i // num_colunas, i % num_colunas].set_title(f'Gráfico de Dispersão - Semente: {semente}')

# Ajustar layout e exibir os subplots
plt.tight_layout()
plt.show()

Aparentemente o tempo de voo planejado não tem interferencia nos atrasos na aterrisagem.

### Categóricas

Nesta etapa da análise exploratória, vamos investigar como as variáveis categóricas selecionadas impactam nos atrasos na aterrissagem.

#### OP_CARRIER

In [ ]:
# Obtendo o número de companhias presentes na base de dados
unique_op_carrier = df_final.select("OP_CARRIER").distinct()

# Transformando em um pandas dataframe
unique_op_carrier = unique_op_carrier.toPandas()

# Visualizando resultado
unique_op_carrier

Existem 22 companhias areareas distintas na base de dados.from pyspark.sql.functions import col, mean, stddev

In [ ]:
# Importando funções 
from pyspark.sql.functions import col, mean, stddev

In [ ]:
# Agrupando e calculando media e desvio padrao
op_carrier_delay_mean = df_final.groupBy("OP_CARRIER").agg(
    mean(col("ARR_DELAY")).alias("media"),
    stddev(col("ARR_DELAY")).alias("desvio_padrao")
)

# Transfromando em um dataframe pandas
op_carrier_delay_mean = op_carrier_delay_mean.toPandas()

# Visualizando
op_carrier_delay_mean

In [ ]:
# Criar o gráfico de barra usando o Seaborn
plt.figure(figsize=(8, 6))  # Tamanho da figura (opcional)
sns.barplot(x='OP_CARRIER', y='media', data=op_carrier_delay_mean.sort_values("media", ascending=False), palette='viridis')

# Adicionar título e rótulos aos eixos
plt.title('Média de Atraso')
plt.xlabel('Companhia Aerea')
plt.ylabel('Atraso na Aterrisagem')

# Exibir o gráfico
plt.show()

Como isso vemos que a companhia com maior média de atraso é a G4.

Devido à presença de muitos valores zerados e negativos, os desvios padrões apresentaram valores consideravelmente elevados. A fim de adotar uma abordagem que atenue esse efeito, será conduzido o mesmo cálculo, porém, com a aplicação de um filtro nos atrasos. Dessa forma, obteremos as companhias aéreas com os maiores valores de atraso.

In [ ]:
# Agrupando e calculando media e desvio padrao
op_carrier_delay_mean_2 = df_final.filter(col("ARR_DELAY").isNotNull() & (col("ARR_DELAY") > 0)).groupBy("OP_CARRIER").agg(
    mean(col("ARR_DELAY")).alias("media"),
    stddev(col("ARR_DELAY")).alias("desvio_padrao")
)

# Transfromando em um dataframe pandas
op_carrier_delay_mean_2 = op_carrier_delay_mean_2.toPandas()

# Visualizando
op_carrier_delay_mean_2

In [ ]:
# Criar o gráfico de barra usando o Seaborn
plt.figure(figsize=(8, 6))  # Tamanho da figura (opcional)
sns.barplot(x='OP_CARRIER', y='media', data=op_carrier_delay_mean_2.sort_values("media", ascending=False), palette='viridis')

# Adicionar título e rótulos aos eixos
plt.title('Média dos voos atrasados')
plt.xlabel('Companhia Aerea')
plt.ylabel('Atraso na Aterrisagem')

# Exibir o gráfico
plt.show()

#### ORIGIN

Obtendo os aeroportos de partida com maior atraso médio.

In [ ]:
# Agrupando e calculando media e desvio padrao
origin_delay_mean = df_final.groupBy("ORIGIN").agg(
    mean(col("ARR_DELAY")).alias("media"),
    stddev(col("ARR_DELAY")).alias("desvio_padrao")
)

# Transfromando em um dataframe pandas
origin_delay_mean = origin_delay_mean.toPandas()

# Visualizando
origin_delay_mean

Como temos um número elevado de aeroportos, iremos ánalisar os top10 aeroportos com maior e menor média de atraso.

In [ ]:
# Criar o gráfico de barra usando o Seaborn
plt.figure(figsize=(8, 6))  # Tamanho da figura (opcional)
sns.barplot(x='ORIGIN', y='media', data=origin_delay_mean.sort_values("media", ascending=False).head(10), color='red')

# Adicionar título e rótulos aos eixos
plt.title('Top 10 aeroportos de origem com maior média de atraso')
plt.xlabel('Aeroporto de Origem')
plt.ylabel('Atraso na Aterrisagem')

# Exibir o gráfico
plt.show()

In [ ]:
# Criar o gráfico de barra usando o Seaborn
plt.figure(figsize=(8, 6))  # Tamanho da figura (opcional)
sns.barplot(x='ORIGIN', y='media', data=origin_delay_mean.sort_values("media", ascending=False).tail(10), color='blue')

# Adicionar título e rótulos aos eixos
plt.title('Top 10 aeroportos de origem com menor média de atraso')
plt.xlabel('Aeroporto de Origem')
plt.ylabel('Atraso na Aterrisagem')

# Exibir o gráfico
plt.show()

#### DEST

Nesta etapa iremos analisar os aeroportos que mais recebem voos atrasados com base na média.

In [ ]:
# Agrupando e calculando media e desvio padrao
dest_delay_mean = df_final.groupBy("DEST").agg(
    mean(col("ARR_DELAY")).alias("media"),
    stddev(col("ARR_DELAY")).alias("desvio_padrao")
)

# Transfromando em um dataframe pandas
dest_delay_mean = dest_delay_mean.toPandas()

# Visualizando
dest_delay_mean

In [ ]:
# Criar o gráfico de barra usando o Seaborn
plt.figure(figsize=(8, 6))  # Tamanho da figura (opcional)
sns.barplot(x='DEST', y='media', data=dest_delay_mean.sort_values("media", ascending=False).head(10), color='red')

# Adicionar título e rótulos aos eixos
plt.title('Top 10 aeroportos que recebem voos atrasados com base na média')
plt.xlabel('Aeroporto de Destino')
plt.ylabel('Atraso na Aterrisagem')

# Exibir o gráfico
plt.show()

In [ ]:
# Criar o gráfico de barra usando o Seaborn
plt.figure(figsize=(8, 6))  # Tamanho da figura (opcional)
sns.barplot(x='DEST', y='media', data=dest_delay_mean.sort_values("media", ascending=False).tail(10), color='blue')

# Adicionar título e rótulos aos eixos
plt.title('Tail 10 aeroportos que recebem voos atrasados com base na média')
plt.xlabel('Aeroporto de Origem')
plt.ylabel('Atraso na Aterrisagem')

# Exibir o gráfico
plt.show()

# Modelagem

In [ ]:
# This list includes all values not known at the moment of takeoff
# except `ARR_DELAY` which will be used as target variable
take_off_leak = ["WHEELS_ON","TAXI_IN","ARR_TIME","ACTUAL_ELAPSED_TIME","AIR_TIME",
                 "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]

In [ ]:
take_off_df = df_final.drop(*take_off_leak)\
                      .filter(df_final.CANCELLED == 0)\
                      .filter(df_final.DIVERTED == 0)\
                      .filter(df_final.CRS_ARR_TIME.isNotNull())\
                      .filter(df_final.ARR_DELAY.isNotNull())\
                      .filter(df_final.DEP_DELAY.isNotNull())

In [ ]:
missing_counts = take_off_df.select([sf.col(column).isNull().cast("int").alias(column) for column in take_off_df.columns]) \
                       .groupBy() \
                       .sum()

In [ ]:
# Criando dataframe de colunas com valores zerados
missing_counts_df = missing_counts.toPandas().transpose()

# Filtrando apenas colunas com valores nulos
missing_counts_df = missing_counts_df[missing_counts_df[0]>0]

# Renomeando a coluna
missing_counts_df = missing_counts_df.rename(columns={0:"nulos"})

In [ ]:
# Cálculando porcentagem de valores faltantes
missing_counts_df["%nulos"] = (missing_counts_df["nulos"]/OBSERVACOES) * 100

# Ordenando por % de nulos
missing_counts_df = missing_counts_df.sort_values("%nulos", ascending=False)

# Visualizando resultados
missing_counts_df

## Train/Test Split

In [ ]:
train_df, test_df = take_off_df.randomSplit([0.8,0.2], seed=42)
toy_df = train_df.sample(False, 0.1, seed=42)

In [ ]:
print("Train set count:", train_df.count())
print("Test set count:", test_df.count())
print("Toy set count:", toy_df.count())

## Feature Engineering: One-Hot-Enconding

In [ ]:
cat_features = ["OP_CARRIER", "OP_CARRIER_FL_NUM", "ORIGIN", "DEST"]

indexOutputCols = [x + 'Index' for x in cat_features]

oheOutputCols = [x + 'OHE' for x in cat_features]

stringIndex = StringIndexer(inputCols = cat_features,
                            outputCols = indexOutputCols,
                            handleInvalid = 'skip')

oheEncoder = OneHotEncoder(inputCols = indexOutputCols,
                           outputCols = oheOutputCols)

In [ ]:
num_features = ["TAXI_OUT", "DEP_DELAY", "CRS_ELAPSED_TIME", "DISTANCE"]

numVecAssembler = VectorAssembler(inputCols = num_features,
                                  outputCol = 'features',
                                  handleInvalid = 'skip')

stdScaler = StandardScaler(inputCol = 'features',
                           outputCol = 'features_scaled')

## Assembling dos vetores

In [ ]:
assembleInputs = oheOutputCols + ['features_scaled']

vecAssembler = VectorAssembler(inputCols = assembleInputs,
                               outputCol = 'features_vector')

In [ ]:
stages = [stringIndex, oheEncoder, numVecAssembler, stdScaler, vecAssembler]

## Criação do Pipeline

In [ ]:
# Criacao do pipeline de transformacao
transform_pipeline = Pipeline(stages=stages)

# Aplicacao do pipeline nos dados de treino
fitted_transformer = transform_pipeline.fit(train_df)
transformed_train_df = fitted_transformer.transform(train_df)

transformed_train_df.limit(10).toPandas()

## Model Training

In [ ]:
model = LinearRegression(maxIter = 5, # pode causar overfitting
                         solver = 'auto',
                         labelCol = 'ARR_DELAY',
                         featuresCol = 'features_vector',
                         elasticNetParam = 0.2,
                         regParam = 0.02)

pipe_stages = stages + [model]

pipe = Pipeline(stages=pipe_stages)

In [ ]:
fitted_pipe = pipe.fit(toy_df)

## Model performance evaluation

In [ ]:
preds = fitted_pipe.transform(test_df)

In [ ]:
preds.limit(10).toPandas()

In [ ]:
rmse = RegressionEvaluator(labelCol = 'ARR_DELAY',
                           metricName = 'rmse').evaluate(preds)

In [ ]:
print("RMSEof Prediction on test set:", rmse)

In [ ]:
results = []
results.append({"run": 1,
                "rmse": rmse,
                "model": "LinearRegression",
                "params": "maxIter = 5, solver = 'auto', labelCol = 'ARR_DELAY', featuresCol = 'features_vector', elasticNetParam = 0.2, regParam = 0.02"})

In [ ]:
df_final.drop(*take_off_leak)\
                      .filter(df_final.CANCELLED == 0)\
                      .filter(df_final.DIVERTED == 0)\
                      .filter(df_final.CRS_ARR_TIME.isNotNull())\
                      .filter(df_final.ARR_DELAY.isNotNull())\
                      .filter(df_final.DEP_DELAY.isNotNull()).count()

## Hyperparameter Tuning

In [ ]:
parameter_grid = ParamGridBuilder() \
                   .addGrid(model.elasticNetParam, [0.2, 0.3]) \
                   .addGrid(model.regParam, [0.02, 0.03]) \
                   .build()

cross_val = CrossValidator(estimator=pipe,
                           estimatorParamMaps=parameter_grid,
                           evaluator = RegressionEvaluator(labelCol = 'ARR_DELAY'),
                           numFolds = 5)

cv_model = cross_val.fit(toy_df)

In [ ]:
import pandas as pd

en = [0.2, 0.3]
reg = [0.02, 0.03]

elastic_net = [e for e in en for r in reg]
regularization = [r for e in en for r in reg]

rmse_df = pd.DataFrame({'rmse': cv_model.avgMetrics,
                        'elastic_net_alpha': elastic_net,
                        'regularization_term': regularization})

rmse_df.sort_values(by='rmse')